In [1]:
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

# Parameters for the model and dataset.
TRAINING_SIZE = 50000
DIGITS = 3
REVERSE = True

# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
MAXLEN = DIGITS + 1 + DIGITS

In [8]:
class CharacterTable:
    """Given a set of characters:
    + Encode them to a one-hot integer representation
    + Decode the one-hot or integer representation to their character output
    + Decode a vector of probabilities to their character output
    """

    def __init__(self, chars):
        """Initialize character table.
        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
        print(self.char_indices)
        print('indices_char', self.indices_char)
    def encode(self, C, num_rows):
        """One-hot encode given string C.
        # Arguments
            C: string, to be encoded.
            num_rows: Number of rows in the returned one-hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        """Decode the given vector or 2D array to their character output.
        # Arguments
            x: A vector or a 2D array of probabilities or one-hot representations;
                or a vector of character indices (used with `calc_argmax=False`).
            calc_argmax: Whether to find the character index with maximum
                probability, defaults to `True`.
        """
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[x] for x in x)


# All the numbers, plus sign and space for padding.
chars = "0123456789+ "
ctable = CharacterTable(chars)


{' ': 0, '+': 1, '0': 2, '1': 3, '2': 4, '3': 5, '4': 6, '5': 7, '6': 8, '7': 9, '8': 10, '9': 11}
indices_char {0: ' ', 1: '+', 2: '0', 3: '1', 4: '2', 5: '3', 6: '4', 7: '5', 8: '6', 9: '7', 10: '8', 11: '9'}


In [12]:
f = lambda: int(
    "".join(
        np.random.choice(list("0123456789"))
        for i in range(np.random.randint(1, DIGITS + 1))
    )
)
a, b = f(), f()
a, b

(39, 772)

In [13]:

class CharacterTable:
    """Given a set of characters:
    + Encode them to a one-hot integer representation
    + Decode the one-hot or integer representation to their character output
    + Decode a vector of probabilities to their character output
    """

    def __init__(self, chars):
        """Initialize character table.
        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
#         print(self.char_indices)
#         print('indices_char', self.indices_char)
    def encode(self, C, num_rows):
        """One-hot encode given string C.
        # Arguments
            C: string, to be encoded.
            num_rows: Number of rows in the returned one-hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        """Decode the given vector or 2D array to their character output.
        # Arguments
            x: A vector or a 2D array of probabilities or one-hot representations;
                or a vector of character indices (used with `calc_argmax=False`).
            calc_argmax: Whether to find the character index with maximum
                probability, defaults to `True`.
        """
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[x] for x in x)


# All the numbers, plus sign and space for padding.
chars = "0123456789+ "
ctable = CharacterTable(chars)

questions = []
expected = []
seen = set()
print("Generating data...")
while len(questions) < TRAINING_SIZE:
    f = lambda: int(
        "".join(
            np.random.choice(list("0123456789"))
            for i in range(np.random.randint(1, DIGITS + 1))
        )
    )
    #f() is the function/lambda function that randomly takes a number between 1 to 4
    #and generates a number of that length which is also random
    a, b = f(), f() #generate two distinct integers
    # Skip any addition questions we've already seen
    # Also skip any such that x+Y == Y+x (hence the sorting).
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    # Pad the data with spaces such that it is always MAXLEN.
    q = "{}+{}".format(a, b)
    query = q + " " * (MAXLEN - len(q))
    ans = str(a + b)
    # Answers can be of maximum size DIGITS + 1.
    ans += " " * (DIGITS + 1 - len(ans))
    if REVERSE:
        # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
        # space used for padding.)
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print("Total questions:", len(questions))

{' ': 0, '+': 1, '0': 2, '1': 3, '2': 4, '3': 5, '4': 6, '5': 7, '6': 8, '7': 9, '8': 10, '9': 11}
indices_char {0: ' ', 1: '+', 2: '0', 3: '1', 4: '2', 5: '3', 6: '4', 7: '5', 8: '6', 9: '7', 10: '8', 11: '9'}
Generating data...
Total questions: 50000


In [5]:
print("Vectorization...")
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
# print(indices)
x = x[indices]
y = y[indices]

# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print("Training Data:")
print(x_train.shape)
print(y_train.shape)

print("Validation Data:")
print(x_val.shape)
print(y_val.shape)

Vectorization...
Training Data:
(45000, 7, 12)
(45000, 4, 12)
Validation Data:
(5000, 7, 12)
(5000, 4, 12)


In [18]:
print("Build model...")
num_layers = 1  # Try to add more LSTM layers!

model = keras.Sequential()
# "Encode" the input sequence using a LSTM, producing an output of size 128.
# Note: In a situation where your input sequences have a variable length,
# use input_shape=(None, num_feature).
model.add(layers.LSTM(128, input_shape=(MAXLEN, len(chars))))
# As the decoder RNN's input, repeatedly provide with the last output of
# RNN for each time step. Repeat 'DIGITS + 1' times as that's the maximum
# length of output, e.g., when DIGITS=3, max output is 999+999=1998.
model.add(layers.RepeatVector(DIGITS + 1)) 
#RepeatVector repeats the last vector to given times
#if previous shape is (none, 16) then if RepeatVector(4) is called
#then the shape becomes (none, 4, 16)
# The decoder RNN could be multiple layers stacked or a single layer.
for _ in range(num_layers):
    # By setting return_sequences to True, return not only the last output but
    # all the outputs so far in the form of (num_samples, timesteps,
    # output_dim). This is necessary as TimeDistributed in the below expects
    # the first dimension to be the timesteps.
    model.add(layers.LSTM(128, return_sequences=True))

# Apply a dense layer to the every temporal slice of an input. For each of step
# of the output sequence, decide which character should be chosen.
model.add(layers.Dense(len(chars), activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()


Build model...
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
dense_1 (Dense)              (None, 4, 12)             1548      
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


In [7]:
epochs = 30
batch_size = 32


# Train the model each generation and show predictions against the validation
# dataset.
for epoch in range(1, epochs):
    print()
    print("Iteration", epoch)
    model.fit(
        x_train,
        y_train,
        batch_size=batch_size,
        epochs=1,
        validation_data=(x_val, y_val),
    )
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = np.argmax(model.predict(rowx), axis=-1)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print("Q", q[::-1] if REVERSE else q, end=" ")
        print("T", correct, end=" ")
        if correct == guess:
            print("☑ " + guess)
        else:
            print("☒ " + guess)



Iteration 1
1407/1407 [==============================] - 28s 17ms/step - loss: 1.7471 - accuracy: 0.3594 - val_loss: 1.5369 - val_accuracy: 0.4168
Q 3+19    T 22   ☒ 11  
Q 989+216 T 1205 ☒ 1110
Q 516+961 T 1477 ☒ 1441
Q 977+667 T 1644 ☒ 1441
Q 96+965  T 1061 ☑ 1061
Q 831+40  T 871  ☒ 933 
Q 95+929  T 1024 ☒ 1090
Q 316+781 T 1097 ☒ 1110
Q 419+359 T 778  ☒ 500 
Q 78+92   T 170  ☒ 998 

Iteration 2
1407/1407 [==============================] - 24s 17ms/step - loss: 1.3476 - accuracy: 0.4976 - val_loss: 1.1668 - val_accuracy: 0.5710
Q 392+5   T 397  ☒ 394 
Q 939+89  T 1028 ☒ 1020
Q 58+738  T 796  ☒ 809 
Q 787+55  T 842  ☒ 823 
Q 868+5   T 873  ☒ 880 
Q 44+97   T 141  ☒ 156 
Q 50+32   T 82   ☒ 60  
Q 726+92  T 818  ☒ 810 
Q 948+396 T 1344 ☒ 1390
Q 64+139  T 203  ☑ 203 

Iteration 3
1407/1407 [==============================] - 25s 17ms/step - loss: 1.0535 - accuracy: 0.6098 - val_loss: 0.9743 - val_accuracy: 0.6420
Q 121+6   T 127  ☒ 129 
Q 726+69  T 795  ☒ 797 
Q 36+667  T 703  ☒ 795 
Q 18

1407/1407 [==============================] - 25s 18ms/step - loss: 0.0200 - accuracy: 0.9945 - val_loss: 0.0051 - val_accuracy: 0.9994
Q 9+184   T 193  ☑ 193 
Q 2+509   T 511  ☑ 511 
Q 293+78  T 371  ☑ 371 
Q 1+325   T 326  ☑ 326 
Q 11+411  T 422  ☑ 422 
Q 7+95    T 102  ☑ 102 
Q 409+1   T 410  ☑ 410 
Q 450+213 T 663  ☑ 663 
Q 19+47   T 66   ☑ 66  
Q 613+686 T 1299 ☑ 1299

Iteration 23
1407/1407 [==============================] - 23s 16ms/step - loss: 0.0184 - accuracy: 0.9948 - val_loss: 0.0132 - val_accuracy: 0.9967
Q 452+8   T 460  ☑ 460 
Q 21+975  T 996  ☑ 996 
Q 75+807  T 882  ☑ 882 
Q 912+50  T 962  ☑ 962 
Q 256+27  T 283  ☑ 283 
Q 58+994  T 1052 ☑ 1052
Q 640+797 T 1437 ☑ 1437
Q 87+531  T 618  ☑ 618 
Q 740+879 T 1619 ☑ 1619
Q 18+63   T 81   ☑ 81  

Iteration 24
1407/1407 [==============================] - 23s 16ms/step - loss: 0.0248 - accuracy: 0.9932 - val_loss: 0.0085 - val_accuracy: 0.9983
Q 7+287   T 294  ☑ 294 
Q 317+683 T 1000 ☑ 1000
Q 220+213 T 433  ☑ 433 
Q 6+147   T 153